In [1]:
%load_ext autoreload
%autoreload 2
from utils import*
from preprocessing import preprocessing
from feature_extraction import detectCorners

In [5]:
path_of_the_directory='../data/men/0/'
path_of_the_directory_result='../preprocessing_output/men/0/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

path_of_the_directory='../data/women/0/'
path_of_the_directory_result='../preprocessing_output/women/0/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

In [7]:
path_of_the_directory='../data/men/1/'
path_of_the_directory_result='../preprocessing_output/men/1/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)


path_of_the_directory='../data/women/1/'
path_of_the_directory_result='../preprocessing_output/women/1/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

In [8]:
path_of_the_directory='../data/men/2/'
path_of_the_directory_result='../preprocessing_output/men/2/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)


path_of_the_directory='../data/women/2/'
path_of_the_directory_result='../preprocessing_output/women/2/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

In [12]:
path_of_the_directory='../data/men/3/'
path_of_the_directory_result='../preprocessing_output/men/3/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

path_of_the_directory='../data/women/3/'
path_of_the_directory_result='../preprocessing_output/women/3/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    print(i)
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

In [10]:
path_of_the_directory='../data/men/4/'
path_of_the_directory_result='../preprocessing_output/men/4/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)


path_of_the_directory='../data/women/4/'
path_of_the_directory_result='../preprocessing_output/women/4/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

In [11]:
path_of_the_directory='../data/men/5/'
path_of_the_directory_result='../preprocessing_output/men/5/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)


path_of_the_directory='../data/women/5/'
path_of_the_directory_result='../preprocessing_output/women/5/'
i=0
for filename in os.listdir(path_of_the_directory):
    if(i>20):
        break
    i=i+1
    img = cv2.imread(path_of_the_directory+str( filename))
    if img is None:
        continue

    # Skin Masks
    binary_hand=preprocessing(img, gamma=True, close=True)
    cv2.imwrite(path_of_the_directory_result+str(i)+'.jpg',binary_hand)

Feature Extraction

In [3]:
for i in range(0, 6):
    print(i)
    path_of_the_directory_result_men = '../preprocessing_output/men/' + str(i) + '/'
    path_of_the_directory_result_women = '../preprocessing_output/women/' + str(i) + '/'

    feature_vector_men = []
    feature_vector_women = []

    for filename in os.listdir(path_of_the_directory_result_men):
        print(path_of_the_directory_result_men + str(filename))
        corners_detected = detectCorners(path_of_the_directory_result_men + str(filename))
        feature_vector_men.append(corners_detected)
    
    # export feature_vector_men to excel

    for filename in os.listdir(path_of_the_directory_result_women):
        print(path_of_the_directory_result_women + str(filename))
        corners_detected = detectCorners(path_of_the_directory_result_women + str(filename))
        feature_vector_women.append(corners_detected)
    
    # export feature_vector_women to excel

0
1
../preprocessing_output/men/1/edge_1_men (1).JPG
../preprocessing_output/men/1/edge_1_men (10).JPG
../preprocessing_output/men/1/edge_1_men (100).JPG
../preprocessing_output/men/1/edge_1_men (101).JPG
../preprocessing_output/men/1/edge_1_men (102).JPG
../preprocessing_output/men/1/edge_1_men (103).JPG
../preprocessing_output/men/1/edge_1_men (104).JPG
../preprocessing_output/men/1/edge_1_men (105).JPG
../preprocessing_output/men/1/edge_1_men (106).JPG
../preprocessing_output/men/1/edge_1_men (107).JPG
../preprocessing_output/men/1/edge_1_men (108).JPG
../preprocessing_output/men/1/edge_1_men (109).JPG
../preprocessing_output/men/1/edge_1_men (11).JPG
../preprocessing_output/men/1/edge_1_men (110).JPG
../preprocessing_output/men/1/edge_1_men (111).JPG
../preprocessing_output/men/1/edge_1_men (112).JPG
../preprocessing_output/men/1/edge_1_men (113).JPG
../preprocessing_output/men/1/edge_1_men (114).JPG
../preprocessing_output/men/1/edge_1_men (115).JPG
../preprocessing_output/men/1/e